<a href="https://colab.research.google.com/github/Lauralethia/SCdD_TPs-GIT/blob/master/my_classes_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Definición de clases

In [0]:
import torch
from torch.utils import data

class Dataset(object):
  
  def __init__(self, list_IDs, labels):
    'Initialization'
    self.labels = labels
    self.list_IDs = list_IDs

  def __len__(self):
    'Denotes the total number of samples'
    return len(self.list_IDs)

  def __getitem__(self, index):
    'Generates one sample of data'
    # Select sample
    ID = self.list_IDs[index]

    # Load data and get label
    X = torch.load('data/' + ID + '.pt')
    y = self.labels[ID]

    return X, y


In [0]:
## Carga de Datos del disco
class MRIDataset(object):
    """MRI dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.

        """
        self.labels_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir

    def __len__(self):
        return len(self.labels_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.labels_frame.ix[idx, 0])
        nimg = nib.load(img_name)
        image = nimg.get_data()
        image = np.nan_to_num(image)
        image = nd.interpolation.zoom(image,0.25)
        image = torch.from_numpy(image)
        image = image.unsqueeze(0)
        image = image.unsqueeze(0)
        image = image.float()
        label = self.labels_frame.ix[idx, 1]
        dummy = np.ndarray([1],dtype='float32')
        dummy[0]= label
        label = torch.from_numpy(dummy)
        sample = {'image': image, 'label': label}
    
        return sample 


In [0]:
"""
Created on Sun Dec  3 16:22:39 2017
Toma las imagenes, elimina numeros menores a 0, luego als rota (np.random.rand()-0.5)*10 
y las squizea np.random.rand()-0.5)*5 (esto creo que podriamos considerar eliminarlo) 
@author: 
"""
import numpy as np
from torch.autograd import Variable
from scipy import ndimage as nd
import torch
class dataugment(object):
    """Augment MRI dataset."""

    def __init__(self, inputs, labels):
        self.samples = inputs.data
        self.labels = labels.data
        self.labels_frame = labels
    
    def __len__(self):
        return len(self.labels_frame) 
    
    def __getitem__(self, idx):
        image = self.samples[idx,:,:,:]
        image = Variable(image)
        image = image.data.numpy()
           
        image[image<0] = 0
#        image = image-image[image>0].mean()
#        image = image/(image.std()+1e-3)
            
        image = image.squeeze() 
        angle = (np.random.rand()-0.5)*10  # 10
        image = nd.rotate(image,angle,axes=(0,1),reshape=False)
        angle = (np.random.rand()-0.5)*10  # 10
        image = nd.rotate(image,angle,axes=(1,2),reshape=False)
        rshiftx = (np.random.rand()-0.5)*5 # 3
        rshiftx = np.round(rshiftx,0)
        rshifty = (np.random.rand()-0.5)*5 # 3
        rshifty = np.round(rshifty,0) 
        rshiftz = (np.random.rand()-0.5)*5 # 3
        rshiftz = np.round(rshiftz,0)
        image = nd.shift(image,[rshiftx, rshifty, rshiftz])
        image = torch.from_numpy(image)
        image = image.unsqueeze(0)
        label = self.labels[idx]        
        sample = {'image': image, 'label': label}
        return sample